# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 15 2022 2:35PM,239678,10,0085772532,ISDIN Corporation,Released
1,Jun 15 2022 2:35PM,239678,10,0085772693,ISDIN Corporation,Released
2,Jun 15 2022 2:35PM,239678,10,0085772746,ISDIN Corporation,Released
3,Jun 15 2022 2:35PM,239678,10,0085772747,ISDIN Corporation,Released
4,Jun 15 2022 2:35PM,239678,10,0085772748,ISDIN Corporation,Released
5,Jun 15 2022 2:35PM,239678,10,0085772911,ISDIN Corporation,Released
6,Jun 15 2022 2:35PM,239678,10,0085772910,ISDIN Corporation,Released
7,Jun 15 2022 2:35PM,239678,10,0085772913,ISDIN Corporation,Released
8,Jun 15 2022 2:35PM,239678,10,0085772912,ISDIN Corporation,Released
9,Jun 15 2022 2:35PM,239678,10,0085773035,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
46,239674,Released,1
47,239675,Released,1
48,239677,Released,5
49,239678,Released,37
50,239679,Released,14


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239674,NaN,NaN,1.0
239675,NaN,NaN,1.0
239677,NaN,NaN,5.0
239678,NaN,NaN,37.0
239679,NaN,NaN,14.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239590,0.0,0.0,1.0
239593,0.0,1.0,0.0
239595,0.0,1.0,0.0
239608,9.0,2.0,4.0
239609,3.0,3.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239590,0,0,1
239593,0,1,0
239595,0,1,0
239608,9,2,4
239609,3,3,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239590,0,0,1
1,239593,0,1,0
2,239595,0,1,0
3,239608,9,2,4
4,239609,3,3,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239590,,,1
1,239593,,1,
2,239595,,1,
3,239608,9,2,4
4,239609,3,3,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 15 2022 2:35PM,239678,10,ISDIN Corporation
37,Jun 15 2022 2:25PM,239679,19,"AdvaGen Pharma, Ltd"
51,Jun 15 2022 2:18PM,239677,10,"Methapharm, Inc."
56,Jun 15 2022 2:17PM,239675,10,ISDIN Corporation
57,Jun 15 2022 2:11PM,239674,20,"Exact-Rx, Inc."
58,Jun 15 2022 2:09PM,239672,20,Alumier Labs Inc.
87,Jun 15 2022 2:09PM,239673,19,ACG North America LLC
88,Jun 15 2022 2:05PM,239653,10,ISDIN Corporation
140,Jun 15 2022 2:05PM,239647,10,ISDIN Corporation
175,Jun 15 2022 2:05PM,239639,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 15 2022 2:35PM,239678,10,ISDIN Corporation,,,37
1,Jun 15 2022 2:25PM,239679,19,"AdvaGen Pharma, Ltd",,,14
2,Jun 15 2022 2:18PM,239677,10,"Methapharm, Inc.",,,5
3,Jun 15 2022 2:17PM,239675,10,ISDIN Corporation,,,1
4,Jun 15 2022 2:11PM,239674,20,"Exact-Rx, Inc.",,,1
5,Jun 15 2022 2:09PM,239672,20,Alumier Labs Inc.,,,29
6,Jun 15 2022 2:09PM,239673,19,ACG North America LLC,,,1
7,Jun 15 2022 2:05PM,239653,10,ISDIN Corporation,,,52
8,Jun 15 2022 2:05PM,239647,10,ISDIN Corporation,,,35
9,Jun 15 2022 2:05PM,239639,10,ISDIN Corporation,,,43


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 2:35PM,239678,10,ISDIN Corporation,37,,
1,Jun 15 2022 2:25PM,239679,19,"AdvaGen Pharma, Ltd",14,,
2,Jun 15 2022 2:18PM,239677,10,"Methapharm, Inc.",5,,
3,Jun 15 2022 2:17PM,239675,10,ISDIN Corporation,1,,
4,Jun 15 2022 2:11PM,239674,20,"Exact-Rx, Inc.",1,,
5,Jun 15 2022 2:09PM,239672,20,Alumier Labs Inc.,29,,
6,Jun 15 2022 2:09PM,239673,19,ACG North America LLC,1,,
7,Jun 15 2022 2:05PM,239653,10,ISDIN Corporation,52,,
8,Jun 15 2022 2:05PM,239647,10,ISDIN Corporation,35,,
9,Jun 15 2022 2:05PM,239639,10,ISDIN Corporation,43,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 2:35PM,239678,10,ISDIN Corporation,37,,
1,Jun 15 2022 2:25PM,239679,19,"AdvaGen Pharma, Ltd",14,,
2,Jun 15 2022 2:18PM,239677,10,"Methapharm, Inc.",5,,
3,Jun 15 2022 2:17PM,239675,10,ISDIN Corporation,1,,
4,Jun 15 2022 2:11PM,239674,20,"Exact-Rx, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 2:35PM,239678,10,ISDIN Corporation,37.0,NaN,NaN
1,Jun 15 2022 2:25PM,239679,19,"AdvaGen Pharma, Ltd",14.0,NaN,NaN
2,Jun 15 2022 2:18PM,239677,10,"Methapharm, Inc.",5.0,NaN,NaN
3,Jun 15 2022 2:17PM,239675,10,ISDIN Corporation,1.0,NaN,NaN
4,Jun 15 2022 2:11PM,239674,20,"Exact-Rx, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 15 2022 2:35PM,239678,10,ISDIN Corporation,37.0,0.0,0.0
1,Jun 15 2022 2:25PM,239679,19,"AdvaGen Pharma, Ltd",14.0,0.0,0.0
2,Jun 15 2022 2:18PM,239677,10,"Methapharm, Inc.",5.0,0.0,0.0
3,Jun 15 2022 2:17PM,239675,10,ISDIN Corporation,1.0,0.0,0.0
4,Jun 15 2022 2:11PM,239674,20,"Exact-Rx, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3594824,239.0,32.0,31.0
15,718938,8.0,0.0,0.0
16,239651,3.0,0.0,0.0
18,479271,2.0,0.0,0.0
19,1917091,28.0,5.0,12.0
20,1437970,86.0,31.0,15.0
21,479188,0.0,2.0,0.0
22,718977,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3594824,239.0,32.0,31.0
1,15,718938,8.0,0.0,0.0
2,16,239651,3.0,0.0,0.0
3,18,479271,2.0,0.0,0.0
4,19,1917091,28.0,5.0,12.0
5,20,1437970,86.0,31.0,15.0
6,21,479188,0.0,2.0,0.0
7,22,718977,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,239.0,32.0,31.0
1,15,8.0,0.0,0.0
2,16,3.0,0.0,0.0
3,18,2.0,0.0,0.0
4,19,28.0,5.0,12.0
5,20,86.0,31.0,15.0
6,21,0.0,2.0,0.0
7,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,239.0
1,15,Released,8.0
2,16,Released,3.0
3,18,Released,2.0
4,19,Released,28.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Completed,31.0,0.0,0.0,0.0,12.0,15.0,0.0,0.0
Executing,32.0,0.0,0.0,0.0,5.0,31.0,2.0,0.0
Released,239.0,8.0,3.0,2.0,28.0,86.0,0.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Completed,31.0,0.0,0.0,0.0,12.0,15.0,0.0,0.0
1,Executing,32.0,0.0,0.0,0.0,5.0,31.0,2.0,0.0
2,Released,239.0,8.0,3.0,2.0,28.0,86.0,0.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Completed,31.0,0.0,0.0,0.0,12.0,15.0,0.0,0.0
1,Executing,32.0,0.0,0.0,0.0,5.0,31.0,2.0,0.0
2,Released,239.0,8.0,3.0,2.0,28.0,86.0,0.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()